# Marvin query Results

Now that you have performed your first query, let's take at what Marvin returns as a Marvin Results object.

In [12]:
from marvin import config
config.mode='remote'
config.switchSasUrl('local')
config.setRelease('MPL-4')

In [13]:
from marvin.tools.query import Query, Results, doQuery

# make a query
myquery = 'nsa.sersic_mass > 3e11 AND nsa.z < 0.1'
q = Query(searchfilter=myquery)
 
# run a query
r = q.run()

Your parsed filter is: 
and_(nsa.sersic_mass>3e11, nsa.z<0.1)
Results contain of a total of 3, only returning the first 3 results
[u'mangaid', u'plate', u'plateifu', u'name', u'z', u'sersic_mass']


Let's look at the Marvin Results object.  We can see how many results were returned with r.count and r.totalcount

In [17]:
print(r)
print('Total count', r.totalcount)
print('Page count', r.count)

Marvin Results(results=NamedTuple(mangaid=u'1-338090', plate=8131, plateifu=u'8131-3701', name=u'3701', z=0.0806723535060883, sersic_mass=1150581604352.0), 
query=u'SELECT mangadatadb.cube.mangaid AS "cube.mangaid", mangadatadb.cube.plate AS "cube.plate", concat(mangadatadb.cube.plate, \'-\', mangadatadb.ifudesign.name) AS "cube.plateifu", mangadatadb.ifudesign.name AS "ifu.name", mangasampledb.nsa.z AS "nsa.z", mangasampledb.nsa.sersic_mass AS "nsa.sersic_mass" \nFROM mangadatadb.cube JOIN mangadatadb.ifudesign ON mangadatadb.ifudesign.pk = mangadatadb.cube.ifudesign_pk JOIN mangasampledb.manga_target ON mangasampledb.manga_target.pk = mangadatadb.cube.manga_target_pk JOIN mangasampledb.manga_target_to_nsa ON mangasampledb.manga_target.pk = mangasampledb.manga_target_to_nsa.manga_target_pk JOIN mangasampledb.nsa ON mangasampledb.nsa.pk = mangasampledb.manga_target_to_nsa.nsa_pk JOIN mangadatadb.pipeline_info AS drpalias ON drpalias.pk = mangadatadb.cube.pipeline_info_pk \nWHERE mangas

Queries returning more than 1000 results are paginated into chunks of 100.  For anything less than 1000, the query will return everything.  Totalcount shows the total result count, and count shows the returned count in just that page.

The results from your query are stored in the .results attribute, as a list of NamedTuples.  These are like regular tuples except they have names (like dictionary key names)

In [19]:
r.results

[NamedTuple(mangaid=u'1-338090', plate=8131, plateifu=u'8131-3701', name=u'3701', z=0.0806723535060883, sersic_mass=1150581604352.0),
 NamedTuple(mangaid=u'1-377378', plate=8132, plateifu=u'8132-6102', name=u'6102', z=0.0265520866960287, sersic_mass=513574535168.0),
 NamedTuple(mangaid=u'1-256301', plate=8258, plateifu=u'8258-9102', name=u'9102', z=0.0961406528949738, sersic_mass=7116798033920.0)]

You can access specific values of the results through tuple indexing or via the named attribute. 

In [29]:
res = r.results[0]
print('single row', res)
print('mangaid', res[0])
print('mangaid', res.mangaid)

# what are the names
print('names', res.keys())
print(res.sersic_mass)

('single row', NamedTuple(mangaid=u'1-338090', plate=8131, plateifu=u'8131-3701', name=u'3701', z=0.0806723535060883, sersic_mass=1150581604352.0))
('mangaid', u'1-338090')
('mangaid', u'1-338090')
('names', [u'mangaid', u'plate', u'plateifu', u'name', u'z', u'sersic_mass'])
1.15058160435e+12


** But be careful ** Names using the full `table.parameter` syntax cannot be accessed via the named attribute.  This syntax is returned when two parameters with non-unique names are returned, like `ifu.name` and `

In [32]:
r.columns
r.coltoparam

OrderedDict([(u'mangaid', u'mangaid'),
             (u'plate', u'plate'),
             (u'plateifu', u'plateifu'),
             (u'name', u'name'),
             (u'z', u'z'),
             (u'sersic_mass', u'sersic_mass')])